# Wikipedia Q&A Dataset Generator Demo

This notebook demonstrates how to automatically create high-quality question-answer pairs from Wikipedia content using **npcpy** (NPC Compiler for Python).

## What You'll Learn
- How to fetch Wikipedia content programmatically
- How to use npcpy to create AI agents for dataset generation
- How to generate structured Q&A pairs for supervised fine-tuning

## Prerequisites
Make sure you have the required packages installed:
```bash
uv add npcpy wikipedia-api pandas
```

You also need **Ollama** running locally with the Llama 3.2 model:
```bash
ollama pull llama3.2
```

## Step 1: Import Required Libraries

In [ ]:
import wikipedia
import pandas as pd
from npcpy.npc_compiler import NPC

## Step 2: Fetch Wikipedia Content

We'll create a function to fetch Wikipedia content on any topic. This will serve as our source material for generating Q&A pairs.

The function:
- Attempts to fetch the full page content (first 2000 characters)
- Falls back to a summary if the full page isn't available
- Handles errors gracefully

In [ ]:
def fetch_wikipedia_content(topic):
    """Fetch Wikipedia content for a given topic.
    
    Args:
        topic (str): The Wikipedia topic to fetch
        
    Returns:
        str: Wikipedia content (first 2000 chars or summary)
    """
    try:
        page = wikipedia.page(topic)
        return page.content[:2000]  # First 2000 chars
    except:
        return wikipedia.summary(topic, sentences=10)

## Step 3: Create Q&A Dataset with npcpy

Now we'll create the main function that uses **npcpy** to generate question-answer pairs.

### How npcpy Works
- **NPC**: Creates an AI agent with a specific role and personality
- **primary_directive**: Defines what the agent should do
- **model**: Specifies which LLM to use (Llama 3.2 in this case)
- **provider**: Where the model runs (Ollama for local deployment)

The agent will:
1. Take Wikipedia content as input
2. Generate 8 high-quality Q&A pairs
3. Return structured JSON data
4. Save to CSV for training

In [ ]:
import json
import re

def create_qa_dataset_demo(topic="Great Wall of China"):
    """Create a Q&A dataset from Wikipedia content.
    
    Args:
        topic (str): Wikipedia topic to generate Q&A pairs from
        
    Returns:
        pd.DataFrame: DataFrame containing question-answer pairs
    """
    # Fetch Wikipedia content
    wiki_content = fetch_wikipedia_content(topic)
    
    # Create an NPC agent for dataset creation
    data_creator = NPC(
        name='Dataset Creator',
        primary_directive='Create high-quality question-answer pairs from Wikipedia text',
        model='llama3.2:3b',  # Explicitly use 3B model
        provider='ollama'
    )
    
    # Define the expected JSON format
    json_format = '''
    {"pairs": [
        {"question": "When was the Great Wall built?", "answer": "Built from 7th century BC"},
        {"question": "Who joined the walls?", "answer": "Qin Shi Huang"}
    ]}
    '''
    
    # Create the prompt for the LLM
    prompt = f"""From this Wikipedia content, create 8 high-quality question-answer pairs.

Content: {wiki_content}

Each pair needs a specific question and complete answer from the text.
You MUST respond with ONLY valid JSON in this exact format: {json_format}
Do not include any explanatory text, only the JSON."""
    
    # Get response from the LLM (WITHOUT format='json' to avoid npcpy bug)
    response = data_creator.get_llm_response(prompt)
    response_text = response['response']
    
    # Extract JSON from response (handles cases where LLM adds extra text)
    try:
        # Try to find JSON in the response
        json_match = re.search(r'\{.*"pairs".*\}', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            qa_data = json.loads(json_str)
            qa_pairs = qa_data['pairs']
        else:
            # If no JSON found, try parsing the whole response
            qa_data = json.loads(response_text)
            qa_pairs = qa_data['pairs']
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error parsing response: {e}")
        print(f"Raw response: {response_text}")
        return pd.DataFrame()
    
    # Convert to DataFrame and save
    df = pd.DataFrame(qa_pairs)
    df.to_csv('wikipedia_qa_dataset.csv', index=False)
    
    print(f"Created {len(qa_pairs)} Q&A pairs, saved to wikipedia_qa_dataset.csv")
    return df

## Step 4: Run the Demo

Let's generate a Q&A dataset about the Great Wall of China!

You can change the topic to anything you're interested in:
- `create_qa_dataset_demo("Artificial Intelligence")`
- `create_qa_dataset_demo("Quantum Computing")`
- `create_qa_dataset_demo("Machine Learning")`

In [ ]:
# Generate the dataset
qa_dataset = create_qa_dataset_demo()

# Display the results
print("\nGenerated Q&A Pairs:")
print("=" * 80)
qa_dataset

## Next Steps

Now that you have a Q&A dataset, you can:

1. **Generate more datasets**: Run this for multiple topics
2. **Combine datasets**: Merge multiple CSV files for a larger training set
3. **Fine-tune a model**: Use this data for supervised fine-tuning
4. **Evaluate quality**: Review the Q&A pairs and filter low-quality ones

### Try It Yourself

```python
# Generate datasets for multiple topics
topics = ["Python Programming", "Deep Learning", "Natural Language Processing"]

all_datasets = []
for topic in topics:
    df = create_qa_dataset_demo(topic)
    all_datasets.append(df)

# Combine all datasets
combined_df = pd.concat(all_datasets, ignore_index=True)
combined_df.to_csv('combined_qa_dataset.csv', index=False)
```